# Проект для "Викишоп"

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.  

Задача - обучить модель классифицировать комментарии на позитивные и негативные. Для обучения предоставлен набор данных с разметкой о токсичности правок. Метрика качества модели F1 должна быть не меньше 0.75. 

### Содержание

* [1. Подготовка данных](#chapter1)
* [2. Обучение моделей](#chapter2)
* [3. Вывод](#chapter3)

## 1. Подготовка данных <a class="anchor" id="chapter1"></a>

In [1]:
#!python -m spacy download en_core_web_sm

In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.dummy import DummyClassifier
import spacy
from spacy import load
from spacy.lang.en.examples import sentences
from spacy.lang.en import English
import lightgbm as lgb
import warnings

In [3]:
data = pd.read_csv('toxic_comments.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Столбец *Unnamed: 0* дублирует номера строк, удалим его.

In [6]:
data = data.drop(['Unnamed: 0'], axis=1)

Проверим данные на наличие дубликатов:

In [7]:
data.duplicated().sum()

0

Дубликатов нет.  
Проверим данные на дисбаланс.

In [8]:
toxic = data['toxic'].sum()
print(f'Всего записей {data.shape[0]}, из них токсичных {toxic}')

Всего записей 159292, из них токсичных 16186


Присутствует дисбаланс классов. В дальнейшем стоит обратить на это внимание, если не получится добиться желаемых результатов.

Данные загружены.  
Подготовим их.

In [9]:
#небольшая выборка для отладки
#data = data.sample(40000).reset_index(drop=True)

Очистим данные:  
* Избавимся от знаков препинания и чисел, заменив их на пробелы  
* Уберем двойные пробелы  
* Приведем текст к нижнему регистру

In [10]:
data['text_clean'] = data['text'].replace(r'[^\w\s]',' ',regex=True).replace(r'\s+',' ',regex=True).str.lower()
data['text_clean']

0         explanation why the edits made under my userna...
1         d aww he matches this background colour i m se...
2         hey man i m really not trying to edit war it s...
3          more i can t make any real suggestions on imp...
4         you sir are my hero any chance you remember wh...
                                ...                        
159287     and for the second time of asking when your v...
159288    you should be ashamed of yourself that is a ho...
159289    spitzer umm theres no actual article for prost...
159290    and it looks like it was actually you who put ...
159291     and i really don t think you understand i cam...
Name: text_clean, Length: 159292, dtype: object

Лемматизируем текст

In [11]:
nlp = spacy.load('en_core_web_sm')
lemma = []
 
for doc in nlp.pipe(data["text_clean"].values):
    lemma.append([n.lemma_ for n in doc])

data['text_clean_lemma'] = lemma
data[['text_clean','text_clean_lemma']].head()

,text_clean,text_clean_lemma
0,explanation why the edits made under my userna...,"[explanation, why, the, edit, make, under, my,..."
1,d aww he matches this background colour i m se...,"[d, aww, he, match, this, background, colour, ..."
2,hey man i m really not trying to edit war it s...,"[hey, man, I, m, really, not, try, to, edit, w..."
3,more i can t make any real suggestions on imp...,"[ , more, I, can, t, make, any, real, suggesti..."
4,you sir are my hero any chance you remember wh...,"[you, sir, be, my, hero, any, chance, you, rem..."


Уберем стоп-слова и "склеим" лемматизированные слова в строки.

In [12]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\home\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
data['text_clean_lemma'] = data['text_clean_lemma'].apply(lambda x: [item for item in x if item not in stopwords])
data['text_clean_lemma_as_str'] = [' '.join(map(str, l)) for l in data['text_clean_lemma']]
data['text_clean_lemma_as_str']

0         explanation edit make username hardcore metall...
1         aww match background colour I seemingly stuck ...
2         hey man I really try edit war guy constantly r...
3           I make real suggestion improvement I wonder ...
4                             sir hero chance remember page
                                ...                        
159287      second time ask view completely contradict c...
159288    ashamed horrible thing put talk page 128 61 19 93
159289    spitzer umm actual article prostitution ring c...
159290    look like actually put speedy first version de...
159291      I really think understand I come idea bad ri...
Name: text_clean_lemma_as_str, Length: 159292, dtype: object

Получен массив лемматизированных текстов.

## 2. Обучение моделей<a class="anchor" id="chapter2"></a>

Разобьем данные на тренировочную, валидационную и тестовую выборки в отношении 60:20:20.

In [14]:
#features = pd.DataFrame(features)
features_train, features_test, target_train, target_test = train_test_split(data['text_clean_lemma_as_str'], data['toxic'], test_size=0.2)
features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, test_size=0.25)

Вычислим TF-IDF для всех выборок.

In [15]:
tf_idf = TfidfVectorizer(stop_words=stopwords)
features_train = tf_idf.fit_transform(features_train)
features_valid = tf_idf.transform(features_valid)
features_test = tf_idf.transform(features_test)

In [16]:
print(f'features_train {features_train.shape}, target_train {target_train.shape}')
print(f'features_valid {features_valid.shape}, target_valid {target_valid.shape}')
print(f'features_test {features_test.shape}, target_test {target_test.shape}')

features_train (95574, 124279), target_train (95574,)
features_valid (31859, 124279), target_valid (31859,)
features_test (31859, 124279), target_test (31859,)


Рассмотрим модель несколько моделей и выберем лучшую.  
Начнем с логистической регрессии. Т.к. в данных присутствует дисбаланс классов, укажем соответствующий гиперпараметр.

In [17]:
model = LogisticRegression()
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1 = f1_score(target_valid, predicted_valid)
print(f'F1 без учета дисбаланса равна {f1:.2f}')

model = LogisticRegression(class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1 = f1_score(target_valid, predicted_valid)
print(f'F1 с учетом дисбаланса равна {f1:.2f}')

F1 без учета дисбаланса равна 0.73
F1 с учетом дисбаланса равна 0.74


C:\Users\home\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Получили хороший результат, учет дисбаланса положительно сказался на качестве.  
Проверим модель градиентного бустинга.

In [18]:
warnings.filterwarnings('ignore')
print('Результаты с учетом дисбаланса')

for lrate in [0.1, 0.5]:
    for num_iters in [100, 200, 300]:
        param = {
            'objective': 'binary',
            'metric':'binary_error',
            'is_unbalance':True,
            'num_iterations':num_iters, 
            'learning_rate':lrate, 
            "verbose":-100
        }
        model = lgb.LGBMClassifier(**param)
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        print(f"Learning_rate {lrate}, Num_iterations {num_iters}, F1 равна {f1:.2f}")

print('Результаты без учета дисбаланса')

for lrate in [0.1, 0.5]:
    for num_iters in [100, 200, 300]:
        param = {
            'objective': 'binary',
            'metric':'binary_error',
            'is_unbalance':False,
            'num_iterations':num_iters, 
            'learning_rate':lrate, 
            "verbose":-100
        }
        model = lgb.LGBMClassifier(**param)
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        print(f"Learning_rate {lrate}, Num_iterations {num_iters}, F1 равна {f1:.2f}")

Результаты с учетом дисбаланса
Learning_rate 0.1, Num_iterations 100, F1 равна 0.72
Learning_rate 0.1, Num_iterations 200, F1 равна 0.73
Learning_rate 0.1, Num_iterations 300, F1 равна 0.74
Learning_rate 0.5, Num_iterations 100, F1 равна 0.74
Learning_rate 0.5, Num_iterations 200, F1 равна 0.75
Learning_rate 0.5, Num_iterations 300, F1 равна 0.76
Результаты без учета дисбаланса
Learning_rate 0.1, Num_iterations 100, F1 равна 0.75
Learning_rate 0.1, Num_iterations 200, F1 равна 0.77
Learning_rate 0.1, Num_iterations 300, F1 равна 0.77
Learning_rate 0.5, Num_iterations 100, F1 равна 0.76
Learning_rate 0.5, Num_iterations 200, F1 равна 0.76
Learning_rate 0.5, Num_iterations 300, F1 равна 0.76


Результат аналогичен логистической регрессии.  
Возьмем в качестве гиперпараметров значения `Learning_rate` = 0.1, `Num_iterations` = 200.  
Дисбаланс классов учитывать не будем, т.к. в этом случае результат модели хуже.  

In [19]:
best_result = 0
best_est = 0
best_depth = 0
for depth in range(2,7):
    for est in range(10, 51, 10):
        model = RandomForestClassifier(class_weight='balanced', n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        result = f1_score(target_valid, predicted_valid)
        if result > best_result:
            best_result = result
            best_est = est
            best_depth = depth
        
print(f'Лучший результат {best_result:.2f}, количество деревьев {best_est}, глубина дерева {best_depth}')

Лучший результат 0.33, количество деревьев 30, глубина дерева 5


Результат неудовлетворительный.

Модель логистической регрессии показала результат не хуже, чем у градиентного бустинга. Остановимся на ней.  
Проверим модель логистической регрессии на тестовой выборке.

In [20]:
model = LogisticRegression(class_weight='balanced')
model.fit(features_train, target_train)
predicted_test = model.predict(features_test)
f1 = f1_score(target_test, predicted_test)
print(f'F1 с учетом дисбаланса равна {f1:.2f}')

F1 с учетом дисбаланса равна 0.75


Проверим модель на адекватность.  
Чтобы учесть дисбаланс, выберем стратегию `stratified`, 

In [21]:
model = DummyClassifier(strategy='stratified')
model.fit(features_train, target_train)
predicted_test = model.predict(features_test)
f1 = f1_score(target_test, predicted_test)
print(f'F1 с учетом дисбаланса равна {f1:.2f}')

F1 с учетом дисбаланса равна 0.10


Метрика ухудшилась, значит наша модель эффективнее случайных "угадываний".

## 3. Вывод<a class="anchor" id="chapter3"></a>

Удалось получить модель с качеством, удовлетворяющим условиям задачи.  
Исходные данные были обработаны:
- удалены знаки препинания, числа, лишние пробелы,
- удалены стоп-слова (предлоги, местоимения, междометия и другие слова, не несущие большой смысловой нагрузки),
- тексты были лемматизированы.  

Были вычеслены величины TF-IDF для корпуса текстов. Эти величины были использованы в качестве признаков для обучения моделей классификации.  

Было рассмотрено несколько моделей с различными гиперпараметрами: логистическая регрессия, градиентный бустинг и случайный лес.  
Логистическая регрессия и градиентный бустинг показали похожий результат, но выбор сделан в пользу первого из-за скорости вычислений. Модель случайного леса дала неудовлетворительный результат.  
В итоге, модель логистической регрессии подтвердила результат на тестовых данных и прошла проверку на адекватность.  
Итоговая метрика качества F1 равна 0.75.